In [1]:
%load_ext autoreload
%autoreload 2

from aptrade.strategy import BreakoutStrategy

/home/vcaldas/aptrade/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
import sys
sys.path.append('../src')
%load_ext autoreload
%autoreload 2

import os
from polygon import RESTClient
from dotenv import load_dotenv, dotenv_values
load_dotenv()

config = {
    **dotenv_values("../.env"),  # load shared development variables
    # **dotenv_values(".env.secret"),  # load sensitive variables
    # **os.environ,  # override loaded values with environment variables
}

print("Using Polygon API Key:", config.get("POLYGON_API_KEY"))
client = RESTClient(config.get("POLYGON_API_KEY")) # POLYGON_API_KEY is used

aggs = client.get_aggs(
    "VVOS",
    1,
    "minute",

    "2025-06-30",
    "2025-07-01",
)
print(aggs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using Polygon API Key: XtppF2Lh0hKqwozkZdUpJpiaPYdPbDlO
[Agg(open=3.2, high=3.2, low=3.2, close=3.2, volume=361, vwap=3.2039, timestamp=1751279640000, transactions=9, otc=None), Agg(open=3.2, high=3.2, low=3.2, close=3.2, volume=191, vwap=3.2, timestamp=1751279880000, transactions=4, otc=None), Agg(open=3.35, high=3.35, low=3.35, close=3.35, volume=750, vwap=3.35, timestamp=1751283000000, transactions=2, otc=None), Agg(open=3.38, high=3.38, low=3.38, close=3.38, volume=524, vwap=3.3802, timestamp=1751285040000, transactions=2, otc=None), Agg(open=3.3312, high=3.3312, low=3.3312, close=3.3312, volume=100, vwap=3.3312, timestamp=1751285460000, transactions=1, otc=None), Agg(open=3.33, high=3.33, low=3.33, close=3.33, volume=300, vwap=3.33, timestamp=1751285520000, transactions=2, otc=None), Agg(open=3.2799, high=3.2799, low=3.2799, close=3.2799, volume=250, vwap=3.2776, timestamp=1751286240000, transa

In [3]:
# Convert the 't' (timestamp) column to date if present
import pandas as pd
df = pd.DataFrame(aggs)
df.head()

# Ensure 't' is present and convert to datetime
if 'timestamp' in df.columns:
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    
print(df.index.tz)
if df.index.tz is None:
    df.index = df.index.tz_localize('UTC')
df.index = df.index.tz_convert('US/Eastern')
print(df.index.tz)

# Select and rename columns for Backtesting library
bt_cols = ["open", "high", "low", "close", "volume"]
df = df[bt_cols]
df.columns = ["Open", "High", "Low", "Close", "Volume"]
df.head()

None
US/Eastern


Open    High     Low   Close  Volume
Date                                                             
2025-06-30 06:34:00-04:00  3.2000  3.2000  3.2000  3.2000   361.0
2025-06-30 06:38:00-04:00  3.2000  3.2000  3.2000  3.2000   191.0
2025-06-30 07:30:00-04:00  3.3500  3.3500  3.3500  3.3500   750.0
2025-06-30 08:04:00-04:00  3.3800  3.3800  3.3800  3.3800   524.0
2025-06-30 08:11:00-04:00  3.3312  3.3312  3.3312  3.3312   100.0

In [17]:
from backtesting import Backtest, Strategy
import pandas as pd
from backtesting import Backtest, Strategy
import datetime as dt

class BreakoutStrategyLocal(Strategy):
    open_range_minutes = 10
    premarket_end = dt.time(9, 30)
    exit_minute_bar = dt.time(15, 59)
    risk_percent = 0.01
    stop_loss_pct = 0.30
    profit_target_pct = 0.30
    pull_back_threshold = 0.20  # 20% pullback
    high_move_threshold = 1.00  # 100% move
    reclaim_pct = 0.90  # 90% reclaim

    def init(self):
        self.current_day = None
        self.prev_close = None
        self.premarket_high = None
        self.premarket_high_time = None
        self.premarket_qualified = False
        self.pullback_detected = False
        self.pullback_low = None
        self.reclaim_detected = False
        self.entry_price = None
        self.traded_today = False

    def _reset_day(self, day, prev_close):
        self.current_day = day
        self.prev_close = prev_close
        self.premarket_high = None
        self.premarket_high_time = None
        self.premarket_qualified = False
        self.pullback_detected = False
        self.pullback_low = None
        self.reclaim_detected = False
        self.entry_price = None
        self.traded_today = False

    def next(self):
        t = self.data.index[-1]
        current_bar_date = t.date()
        current_time = t.time()

        # Detect new day and get previous close
        if self.current_day != current_bar_date:
            # Find previous day's last close
            prev_day_mask = self.data.index.date < current_bar_date
            if prev_day_mask.any():
                self.prev_close = self.data.Close[prev_day_mask][-1]
            else:
                self.prev_close = None
            self._reset_day(current_bar_date, self.prev_close)
            print(f"New day: {current_bar_date}, prev close: {self.prev_close}")

        # 1. Track pre-market high and check for >100% move
        if current_time < self.premarket_end and self.prev_close:
            if self.premarket_high is None or self.data.High[-1] > self.premarket_high:
                self.premarket_high = self.data.High[-1]
                self.premarket_high_time = t
            move = (self.premarket_high - self.prev_close) / self.prev_close
            if move >= self.high_move_threshold:
                self.premarket_qualified = True
                print(f"{current_time} | {self.data.High[-1]} : Premarkety high {self.premarket_high:.2f} is {move*100:.2f}% above prev close")

        # 2. After 100% move, look for 20% pullback from high
        if self.premarket_qualified and not self.pullback_detected:
            # Only consider prices after premarket high
            if self.premarket_high_time and t > self.premarket_high_time:
                pullback = (self.premarket_high - self.data.Low[-1]) / self.premarket_high
                if pullback >= self.pull_back_threshold:
                    self.pullback_detected = True
                    self.pullback_low = self.data.Low[-1]
                    print(f"Pullback detected: {pullback*100:.2f}% from high {self.premarket_high:.2f} to low {self.pullback_low:.2f}")

        # 3. Wait for reclaim to 90% of high (entry trigger)
        if self.pullback_detected and not self.reclaim_detected:
            reclaim_price = self.premarket_high * self.reclaim_pct
            if self.data.Close[-1] >= reclaim_price:
                self.reclaim_detected = True
                self.entry_price = reclaim_price
                print(f"Reclaim detected: price {self.data.Close[-1]:.2f} >= {reclaim_price:.2f}")

        # 4. Enter short if not already in position and not traded today
        if self.reclaim_detected and not self.position and not self.traded_today:
            stop_loss = self.entry_price * (1 + self.stop_loss_pct)
            take_profit = self.entry_price * (1 - self.profit_target_pct)
            per_share_risk = abs(self.entry_price - stop_loss)
            position_size = int((self.risk_percent * self.equity) / per_share_risk)
            print(f"Short entry at {self.entry_price:.2f}, stop {stop_loss:.2f}, tp {take_profit:.2f}, size {position_size}")
            self.sell(size=position_size, tp=take_profit, sl=stop_loss)
            self.traded_today = True

        # 5. Close at end of day
        if self.position and current_time == self.exit_minute_bar:
            print("Closing position at end of day")
            self.position.close()

# Fix: select columns as a list of separate strings
# testDF = df[["open", "high", "low", "close", "volume"]]

def per_share_commission(size, price):
    return abs(size) * 0.0005

# 3. create a backtest, pass in data and the strategy you want to run on the data
bt = Backtest(df, BreakoutStrategyLocal, cash=25_000, commission=per_share_commission, margin=0.25)

# run the strategy
stats = bt.run()
bt.plot()
# print(stats)

# stats['_trades'].to_csv('trade_details.csv', index=False)



New day: 2025-06-30, prev close: None
New day: 2025-07-01, prev close: 3.1793
08:34:00 | 6.6 : Premarkety high 6.60 is 107.59% above prev close
08:35:00 | 5.53 : Premarkety high 6.60 is 107.59% above prev close
Pullback detected: 23.48% from high 6.60 to low 5.05
08:36:00 | 5.38 : Premarkety high 6.60 is 107.59% above prev close
08:37:00 | 5.87 : Premarkety high 6.60 is 107.59% above prev close
08:38:00 | 5.44 : Premarkety high 6.60 is 107.59% above prev close
08:39:00 | 5.5 : Premarkety high 6.60 is 107.59% above prev close
08:40:00 | 6.12 : Premarkety high 6.60 is 107.59% above prev close
Reclaim detected: price 5.98 >= 5.94
Short entry at 5.94, stop 7.72, tp 4.16, size 140
08:41:00 | 6.07 : Premarkety high 6.60 is 107.59% above prev close
08:42:00 | 6.46 : Premarkety high 6.60 is 107.59% above prev close
08:43:00 | 6.3 : Premarkety high 6.60 is 107.59% above prev close
08:44:00 | 6.65 : Premarkety high 6.65 is 109.17% above prev close
08:45:00 | 6.19 : Premarkety high 6.65 is 109.17

/home/vcaldas/aptrade/.venv/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p3621', ...)

In [18]:
stats


Start                     2025-06-30 06:34...
End                       2025-07-01 19:59...
Duration                      1 days 13:25:00
Exposure Time [%]                     3.83562
Equity Final [$]                     25257.74
Equity Peak [$]                      25257.74
Commissions [$]                          0.14
Return [%]                            1.03096
Buy & Hold Return [%]                19.05937
Return (Ann.) [%]                   264.13669
Volatility (Ann.) [%]                     NaN
CAGR [%]                            424.82985
Sharpe Ratio                              NaN
Sortino Ratio                             inf
Calmar Ratio                       1132.69451
Alpha [%]                             1.09224
Beta                                 -0.00322
Max. Drawdown [%]                    -0.23319
Avg. Drawdown [%]                    -0.15216
Max. Drawdown Duration        0 days 00:13:00
Avg. Drawdown Duration        0 days 00:08:00
# Trades                          